# Notebook for text extraction on image
Inga Ulusoy, SSC, July 2022

In [ ]:
# if running on google colab
# flake8-noqa-cell
import os

if "google.colab" in str(get_ipython()):
    # update python version
    # install setuptools
    !pip install setuptools==61 -qqq
    # install misinformation
    !pip install git+https://github.com/ssciwr/misinformation.git -qqq
    # mount google drive for data and API key
    from google.colab import drive

    drive.mount("/content/drive")

In [ ]:
import os
from IPython.display import Image, display
import misinformation
from misinformation import utils as mutils
from misinformation import display as mdisplay
import tensorflow as tf

In [ ]:
# download the models if they are not there yet
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

In [ ]:
images = mutils.find_files(path="data", limit=10)

In [ ]:
for i in images:
    display(Image(filename=i))

In [ ]:
mydict = mutils.initialize_dict(images)

# google cloud vision API
First 1000 images per month are free.

## Inspect the elements per image

In [ ]:
mdisplay.explore_analysis(mydict, identify="text-on-image")

## Or directly analyze for further processing

In [ ]:
for key in mydict:
    print(key)
    mydict[key] = misinformation.text.TextDetector(
        mydict[key], analyse_text=True
    ).analyse_image()

## Convert to dataframe and write csv

In [ ]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

In [ ]:
# check the dataframe
df.head(10)

In [ ]:
# Write the csv
df.to_csv("./data_out.csv")

# Topic analysis
The topic analysis is carried out using [BERTopic](https://maartengr.github.io/BERTopic/index.html) using an embedded model through a [spaCy](https://spacy.io/) pipeline.

BERTopic takes a list of strings as input. The more items in the list, the better for the topic modeling. If the below returns an error for `analyse_topic()`, the reason can be that your dataset is too small.
### Option 1: Use the dictionary as obtained from the above analysis.

In [ ]:
# make a list of all the text_english entries per analysed image from the mydict variable as above
topic_model, topic_df, most_frequent_topics = misinformation.text.PostprocessText(
    mydict=mydict
).analyse_topic()

### Option 2: Read in a csv
Not to analyse too many images on google Cloud Vision, use the csv output to obtain the text (when rerunning already analysed images).

In [ ]:
input_file_path = "data_out.csv"
topic_model, topic_df, most_frequent_topics = misinformation.text.PostprocessText(
    use_csv=True, csv_path=input_file_path
).analyse_topic(return_topics=10)

### Access frequent topics
A topic of `-1` stands for an outlier and should be ignored. Topic count is the number of occurence of that topic. The output is structured from most frequent to least frequent topic.

In [ ]:
print(topic_df)

### Get information for specific topic
The most frequent topics can be accessed through `most_frequent_topics` with the most occuring topics first in the list.

In [ ]:
for topic in most_frequent_topics:
    print("Topic:", topic)

### Topic visualization
The topics can also be visualized. Careful: This only works if there is sufficient data (quantity and quality).

In [ ]:
topic_model.visualize_topics()

### Save the model
The model can be saved for future use.

In [ ]:
topic_model.save("misinfo_posts")